In [44]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import re
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display


In [41]:
filename = 'all_text.csv'
all_text = pd.read_csv(filename)
all_text.token = all_text.token.apply(lambda x: x.strip("[]").replace("'","").split(", "))


In [118]:
def find_word_with_context(document=all_text.filename, search="pain", start=50, end=50):
    doc = all_text.index[all_text.filename == document]
    words = all_text.text[doc].reset_index().text[0]
    res = []
    for m in re.finditer(search, words):
        word_w_context = all_text.text[0][m.start()-start: m.end()+end]
        res.append(word_w_context.replace("\n", ""))
    return res

In [119]:
interact(find_word_with_context, search_word="pelvic");

interactive(children=(Dropdown(description='document', options=('C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'…

In [111]:
# doc = all_text.index[all_text.filename == "C1"]
# words = all_text.text[doc].reset_index().text[0]
# words